# MIDI Feature Extraction

In [1]:
import librosa
from glob import glob
import sys
import os

sys.path.append(os.path.abspath("../"))

In [2]:
from src.datasets import MMD_audio_matches, MMD_md5_metainfo

2022-06-16 04:09:20,908 - datasets.py - INFO - Loading MMD_audio_matched_genre data
2022-06-16 04:09:21,253 - datasets.py - INFO - Loading MMD_md5_metainfo data
2022-06-16 04:09:21,413 - datasets.py - INFO - Loading MMD_audio_matches data


In [5]:
from src.utils import get_md5_from_path, get_midi_path_from_md5, get_midi_path_from_md5_with_check, env

get_midi_path_from_md5(MMD_audio_matches.md5.values[0])

'/data1/meta_midi_dataset/MMD_MIDI/9/7/7/977349d0bec3fed4bd2bef1b57c597d4.mid'

In [8]:
audio_files = glob(os.path.join(env["DATASET_PATH"], "spotify_sample", "**", "*.mp3"), recursive=True)
sids = [p.split(os.path.sep)[-1].replace(".mp3", "") for p in audio_files]
sids[:5]

['65yDyFGWY1nAAEIoPoGSL6',
 '65ygse6Fdnez361FitMvHB',
 '65yMioAFRtmChAuc8WUFBM',
 '65yCuO1rnfwmaX69TvlbYl',
 '65ynkEZJeQskFFZRoqmaLp']

In [13]:
MMD_audio_matches

,md5,score,sid
0,977349d0bec3fed4bd2bef1b57c597d4,0.554720,5n3es2C6R47r4WGfmYL9vZ
1,a8ab220d3e771f14994bcb6104a6e733,0.910470,65yDyFGWY1nAAEIoPoGSL6
2,a8ab220d3e771f14994bcb6104a6e733,0.743186,6WRwxMiwig6czraGVt3xEB
3,a8ab220d3e771f14994bcb6104a6e733,0.775063,6X72YoEcN8Iuw4e70NMb1V
4,a8ab220d3e771f14994bcb6104a6e733,0.675244,1Yg4ZDCR9z6AEHwEuWqNAe
...,...,...,...
10796552,055b5942bd76781f7b15f6e22191e8d7,0.533297,1wbBb6ptQaNsesydMNq3Gs
10796553,0e8a0136092a5f661b8c659808f57c13,0.563319,0Dm8i6nlNyElUTagyRPeiP
10796554,0e8a0136092a5f661b8c659808f57c13,0.567955,5FSjUWgBaezzEbsft1S6Iy
10796555,0e8a0136092a5f661b8c659808f57c13,0.542153,0aHgqVczBG5EFiecFeOFzn


In [12]:
MMD_audio_matches[MMD_audio_matches.sid.isin(sids)]

,md5,score,sid
1,a8ab220d3e771f14994bcb6104a6e733,0.910470,65yDyFGWY1nAAEIoPoGSL6
2,a8ab220d3e771f14994bcb6104a6e733,0.743186,6WRwxMiwig6czraGVt3xEB
33,aa752993c389e95bd0b6a00168142308,0.554295,3FEHv0NlGXYzdUwiIaoJY8
52,a9c5968116d7af8301a2e8cfdd5a5d4c,0.570854,68Dv1YMlBfXQzU7KYKWqSY
70,ac45c832a78728aa8822a3df637682c3,0.500152,6YXIx2Xx0KTp8NVyhxAjpw
...,...,...,...
10796529,05c7c17ab46ef96d408eb4aeca1ba61e,0.525340,69lUWH2dqqIk0wdC3AlI7U
10796532,05c7c17ab46ef96d408eb4aeca1ba61e,0.540325,7syWc3SQlklbSdi3WRnnqj
10796533,05c7c17ab46ef96d408eb4aeca1ba61e,0.540325,59PzGioNVVi53eGl77vZtY
10796552,055b5942bd76781f7b15f6e22191e8d7,0.533297,1wbBb6ptQaNsesydMNq3Gs


In [14]:
midi_files = [p for md5 in MMD_audio_matches[MMD_audio_matches.sid.isin(sids)].md5
              if (p := get_midi_path_from_md5_with_check(md5)) is not None]
len(midi_files)

2588752

In [15]:
midi_files[0]

'/data1/meta_midi_dataset/MMD_MIDI/a/8/a/a8ab220d3e771f14994bcb6104a6e733.mid'

In [17]:
import numpy as np
from src.mgeval.mgeval.core import MgevalMetrics, extract_feature

met = MgevalMetrics()
feature = extract_feature(midi_files[0], drums=False)
all_feature_names = list(filter(lambda a: a[0] != "_", dir(MgevalMetrics)))

for fn in all_feature_names:
    print(fn, end=": ")
    res = getattr(met, fn)(feature)
    if type(res) is np.ndarray:
        print(res.shape, res.size, type(res))
    else:
        print(res, type(res))

avg_IOI: 0.3501526088580247 <class 'numpy.float64'>
avg_pitch_shift: 12.338129496402878 <class 'numpy.float64'>
bar_pitch_class_histogram: (26, 12) 312 <class 'numpy.ndarray'>
bar_used_note: 

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices